In [1]:
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
from IPython.core.display import HTML 

# 1. Reading the data

In [2]:
df1 = pd.read_csv('../HPriceKaggleTrain.csv', index_col='Id')
df2 = pd.read_csv('../HPriceKaggleTest.csv', index_col='Id')
df = pd.concat([df1, df2], axis=0, sort=False).reset_index(drop=True)
df.shape

(2919, 80)

# 2. Creating a Data Describtion file

### DATA TYPES:

1. Useless - useless for machine learning algorithms
2. Nominal - truly categorical, labels, groups without order)
3. Binary - dichotomous, a type of nominal scales that contains only two categories
4. Ordinal - categorical groups with order
5. Count - discrete numerical, this type of data can’t be measured but it can be counted
6. Ratio - contituous with an absolute zero, without a temporal component 
7. Interval - don’t have a „true zero“, positive and/or negative numbers without a temporal component 
8. Time - cyclical numbers with a temporal component — continuous
9. Text
10. Image
11. Audio
12. Video

In [3]:
# from txt to excel
ddes = pd.read_fwf('/home/lana/Desktop/DS_blog/Inbox/HPriceKaggle_data_description.txt', sep=' ', header=None)
writer = pd.ExcelWriter('/home/lana/Desktop/DS_blog/InBetween/HPricesDDescribtion_raw.xlsx')
ddes.to_excel(writer, 'Data_Describtion', index=True)
writer.save()

In [4]:
# after some handwork
ddes = pd.read_excel('/home/lana/Desktop/DS_blog/InBetween/HPricesDDescribtion_handwrited.xlsx')
ddes = ddes.dropna(how='all')
ddes.sample()

,Feature,Unit,Type,Descript,Val
159,Exterior1st,-,nominal,Exterior covering on house,WdShing\tWood Shingles


In [5]:
ddes[['Val', 'Value']] = ddes['Val'].str.split('\t', expand=True)
digits = ['ratio', 'count', 'time']
ddes.loc[ddes['Type'].isin(digits), 'Val'] = 123
ddes.loc[ddes['Type'].isin(digits), 'Value'] = 'digits'

In [6]:
ddes = ddes[['Feature', 'Descript',  'Val', 'Value', 'Type', 'Unit']].sort_values(['Feature'])
writer = pd.ExcelWriter('/home/lana/Desktop/DS_blog/InBetween/HPricesDDescribtion.xlsx')
ddes.to_excel(writer, 'Data_Describtion', index=True)
writer.save()

In [7]:
ddes = pd.read_excel('/home/lana/Desktop/DS_blog/InBetween/HPricesDDescribtion.xlsx', index_col=0)
ddes = ddes.reset_index(drop=True)
ddes.sample()

,Feature,Descript,Val,Value,Type,Unit
228,MSSubClass,Identifies the type of dwelling involved in t...,150,1-1/2 STORY PUD - ALL AGES,nominal,-


A Data Description file was created first and contains fewer common information, but it has description for every categorical value.
- 'Feature' - main data set column name
- 'Descript' - feature meaning
- 'Val' - value
- 'Value' - value meaning
- 'Type' - column's data type
- 'Unit' - given unit of value

In [8]:
len(ddes['Feature'].unique())

80

# 3. Creating a Data Dictionary file

In [9]:
desdf = ddes.groupby(['Feature', 'Descript', 'Type', 'Unit'])['Val'].count()
desdf = pd.DataFrame(desdf).reset_index().drop('Val', axis=1).set_index('Feature')
desdf.head(3)

,Descript,Type,Unit
Feature,,,
1stFlrSF,First Floor square feet,cont,square feet
2ndFlrSF,Second floor square feet,cont,square feet
3SsnPorch,Three season porch area in square feet,cont,square feet


In [10]:
len(desdf) == df.shape[1]

True

In [11]:
def DataDictionary(df, desdf):
    """ Create a Data Dictionary. 
    
        Parameters:
            arg_1: data frame (raw data set)
            arg_2: data describtion frame (columns: Feature, Descript, Type, Unit)
        Returns:
            Data Dictionary Frame with following columns:
                'Description' - feature meaning
                '#Unique' - number of unique values in the columns where NaN calculated as value also  
                '%Missing' - % of missing values
                'TopValue' - the most used value  
                '%UsedTop' - % of using top value  
                'Dtype' - column's python data type 
                'Type' - column's data type 
                'Unit' - given unit of value 
    """

    cols = df.columns.tolist()
    l = len(df)
    x, y, z, v, w, xx, yy, zz, ww = [], [], [], [], [], [], [], [], []
    for col in cols:
        s = df[col]
        ll = len(s.unique())
        x.append(ll)
        
        values = s.value_counts().to_frame().reset_index() #number of using for all values
        
        y.append(values.iloc[0,0]) # a name of top value
        nused = values.iloc[0,1] # a number of using 
        yy.append(nused*100/l)
        
        missed = s.isnull().sum() # number of missing values
        z.append(round(missed*100/l, 2))
        
        v.append(str(s.dtype)) # extract a data type
        
        # extract values from data describtion frame
        w.append(desdf['Descript'][col])
        zz.append(desdf['Type'][col])
        ww.append(desdf['Unit'][col])
    
    ddict = pd.DataFrame({'Description':w, '#Unique':x,'%Missing':z, 'TopValue':y, '%UsedTop':yy,   
                         'Dtype':v, 'Type':zz, 'Unit':ww}, index=[cols]).sort_index()
    return ddict

In [12]:
ddict = DataDictionary(df, desdf)
ddict.head()

,Description,#Unique,%Missing,TopValue,%UsedTop,Dtype,Type,Unit
1stFlrSF,First Floor square feet,1083,0.00,864,1.58,int64,cont,square feet
2ndFlrSF,Second floor square feet,635,0.00,0,57.14,int64,cont,square feet
3SsnPorch,Three season porch area in square feet,31,0.00,0,98.73,int64,cont,square feet
Alley,Type of alley access to property,3,93.22,Grvl,4.11,object,nominal,-
BedroomAbvGr,Bedrooms above grade (does NOT include baseme...,8,0.00,3,54.68,int64,discrete,quantity


In [13]:
writer = pd.ExcelWriter('.../HPricesDDict.xlsx')
ddict.to_excel(writer, 'Data_Dictionary', index=True)
writer.save()